> First time use: follow instructions in the README.md file in this directory.


**[PT]** Português

---

**[EN]** English


# Naturalidade


Análise dos valores da informação sobre a naturalidade dos estudantes

---

# Place of Birth

Information related to place of birth of students


## Setup

In [3]:
from timelinknb import get_db
from ucalumni.config import default_db

db_spec = default_db
db = get_db(db_spec)

## Lista de lugares diferentes e número de ocorrências

---

## List of different places with number of occurrences

In [4]:
from timelinknb.pandas import attribute_values

naturalidades = attribute_values('naturalidade',dates_between=('1500-00-00','1990-00-00'))
naturalidades.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11499 entries, Lisboa to Óvoa, Viseu
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   count     11499 non-null  int64 
 1   date_in   11499 non-null  object
 2   date_max  11499 non-null  object
dtypes: int64(1), object(2)
memory usage: 359.3+ KB


### Lugares principais

---

### Main locations

In [5]:
naturalidades.sort_values('count', ascending=False).head(20)


,count,date_in,date_max
value,,,
Lisboa,8784,1537-02-12,1916-07-19
Coimbra,5526,1537-00-00,1915-10-12
Porto,3391,1537-05-30,1917-10-22
Braga,1608,1540-01-21,1914-07-24
Évora,1072,1537-11-22,1910-10-10
Viseu,986,1537-00-00,1912-07-03
Guimarães,980,1537-12-18,1912-07-18
Lamego,972,1537-00-00,1909-10-05
Aveiro,790,1538-04-21,1913-10-13


### Lugares com menos de 5 estudantes

---

### Locations with less than five students

In [6]:
naturalidades[naturalidades['count']<5].sort_values('count', ascending=False).head(20)

,count,date_in,date_max
value,,,
Abrigada,4,1646-10-22,1738-11-04
Paio Mendes,4,1686-10-01,1696-10-01
Santa Eulália de Barrosas,4,1842-10-26,1910-11-16
Santa Eulália da Palmeira,4,1704-10-01,1725-10-01
Santa Eugénia,4,1743-10-01,1787-10-31
"Santa Comba, Seia",4,1746-10-01,1913-07-16
Sanfins do Douro,4,1597-11-22,1891-10-31
"Sande, Lamego",4,1684-10-01,1764-10-01
Samora Correia,4,1698-10-01,1783-10-31


## Identificação de topónimos

---

## Geocoding

https://craftingdh.netlify.app/tutorials/folium/

In [7]:
!pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 1.3 MB/s eta 0:00:00a 0:00:01


In [9]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="fauc1530-1919", timeout=2)

In [114]:
locations = geolocator.geocode("Ribeira de Litém", featuretype='settlement',exactly_one=False, country_codes='pt')
for location in locations:
    print(f"{str(location):>60} {location.raw['importance']:0.2} {location.latitude:0.8} {location.longitude:0.8} {location.raw['class']}:{location.raw['type']}")

TypeError: 'NoneType' object is not iterable

In [55]:
places = naturalidades[naturalidades['count']<5].sort_values('count', ascending=False).index
places


Index(['Abrigada', 'Paio Mendes', 'Santa Eulália de Barrosas',
       'Santa Eulália da Palmeira', 'Santa Eugénia', 'Santa Comba, Seia',
       'Sanfins do Douro', 'Sande, Lamego', 'Samora Correia',
       'Salvaterra do Extremo',
       ...
       'Granja de Gabrielos', 'Granja de Facalamim',
       'Granja de Espinho, Coimbra', 'Granja de Caria',
       'Granja de Alpiate, Lisboa', 'Granja de Alfarelos',
       'Granja da Azinhaga', 'Granja Nova, Alcobaça', 'Grandola, Lisboa',
       'Óvoa, Viseu'],
      dtype='object', name='value', length=9902)

In [90]:
from os.path import exists
import pandas as pd

cache_file = '../inferences/places/osm-places.csv'
places_cache_cols = ['naturalidade','osm-id','osm-address','osm-importance','osm-class','osm-type','lat','long']

if exists(cache_file):
    places_cache = pd.read_csv(cache_file)
else:
   places_cache = pd.DataFrame(columns=places_cache_cols)

places_cache.head()

,naturalidade,osm-id,osm-address,osm-importance,osm-class,osm-type,lat,long
0,Abrigada,1933544852,"Abrigada, Alenquer, Lisboa, 2580-097, Portugal",0.385000,place,village,39.145073,-9.020676
1,Abrigada,403833380,"Abrigada, Relíquias, Odemira, Beja, Portugal",0.310000,place,isolated_dwelling,37.712690,-8.563345
2,Paio Mendes,5527659136,"Paio Mendes, Nossa Senhora do Pranto, Ferreira...",0.450000,place,hamlet,39.742834,-8.289056
3,Santa Eugénia,4204532,"Santa Eugénia, Alijó, Vila Real, Portugal",0.462719,boundary,administrative,41.348296,-7.410860
4,Santa Eugénia,3809325464,"Santa Eugénia, Bela, Monção, Viana do Castelo,...",0.450000,place,neighbourhood,42.059662,-8.418403


In [91]:
import time

# abide to https://operations.osmfoundation.org/policies/nominatim/


country='pt'

cached = set(places_cache['naturalidade'])

for place in places:
    if place in cached:
        print(place, "cached.")
        continue
    
    if 'Brasil' in place:
        country = 'br'
    else:
        country = 'pt'
        
    cache = (place,None, None, None,None,None,None,None)
    locations = geolocator.geocode(f"{place}", featuretype='settlement',exactly_one=False, country_codes=country)
    if locations is None:
        print("Not found:",place)
    else:
        print(f"Found: {place} ({len(locations)})")
        for location in [loc for loc in locations]:
            if location.raw['class'] in ['place', 'boundary']:
                print(f"{str(location):<60} {location.raw['importance']:0.2} {location.latitude:0.8} {location.longitude:0.8} {location.raw['class']}:{location.raw['type']}")
                osm_id = location.raw['osm_id']
                osm_address = location.address
                osm_importance = location.raw['importance']
                osm_class = location.raw['class']
                osm_type = location.raw['type']
                lat = location.latitude
                long = location.longitude
                cache = (place,osm_id, osm_address, osm_importance,osm_class,osm_type,lat,long)

            else:
                print("   ",location.raw['class'])
    print()
    places_cache = pd.concat([places_cache,pd.DataFrame([cache],columns=places_cache_cols)],axis=0)

    time.sleep(2)


Abrigada cached.
Paio Mendes cached.
Not found: Santa Eulália de Barrosas

Not found: Santa Eulália da Palmeira

Santa Eugénia cached.
Santa Comba, Seia cached.
Sanfins do Douro cached.
Sande, Lamego cached.
Samora Correia cached.
Salvaterra do Extremo cached.
Salto cached.
Salgueiral cached.
Safara cached.
Found: Roma (1)
    natural

Rojão Pequeno cached.
Roda cached.
Not found: Rio das Mortes

Rio Seco cached.
Rio Mau cached.
Ribeiro cached.
Not found: Ribeira de Litém

Not found: Santa Maria de Fregim

Santa Maria do Bouro cached.
Santa Maria do Freixo cached.
Seixas cached.
Sobral do Campo cached.
Sobral Pichorro cached.
Not found: Silhada

Setã cached.
Sernande cached.
Not found: Seras

Sequeiros cached.
Seixo de Gatões cached.
Seixal cached.
Santa Maria do Telhado cached.
Secarias cached.
Sardoura cached.
Santos, São Paulo, Brasil cached.
Not found: Santiago de Custóias

Santa Valha cached.
Santa Tecla cached.
Santa Marinha, Guarda cached.
Not found: Santa Marinha de Arcozelo

R

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Vila+de+Santos&format=json&countrycodes=pt&featuretype=settlement (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=2)"))

In [92]:
places_cache.to_csv(cache_file,index=False)
